In [ ]:
!pip install interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.0/758.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 k

In [ ]:
import pandas as pd
# Makes sure we see all columns
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from interpret.glassbox import (LogisticRegression, ClassificationTree)
from interpret import show
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = "/content/drive/MyDrive"
fnames = [x for x in os.listdir(path) if x.startswith("healthcare")]

print(fnames)

['healthcare-dataset-stroke-data.csv']


In [ ]:
class DataLoader():
  def _init_(self):
    self.data = None
    
  def load_dataset(self, path="/content/drive/MyDrive/healthcare-dataset-stroke-data.csv"):
    self.data = pd.read_csv(path)
      
      
  def preprocess_data(self):
    categorical_cols =["gender",
                      "ever_married",
                      "work_type",
                      "Residence_type",
                      "smoking_status"]
    encoded = pd.get_dummies(self.data[categorical_cols],
                            prefix=categorical_cols)
    self.data = pd.concat([encoded, self.data], axis=1)
    self.data.drop(categorical_cols, axis=1, inplace=True)
    
    self.data.bmi = self.data.bmi.fillna(0)
    self.data.drop(["id"], axis=1, inplace=True)
        
  def get_data_split(self):
    x = self.data.iloc[:,:-1]
    y = self.data.iloc[:,-1]
    return train_test_split(x, y, test_size=0.20, random_state=2021)
    
  def oversample(self, x_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='minority')
    x_np = X_train.to_numpy()
    y_np = y_train.to_numpy()
    x_np, y_np = oversample.fit_resample( x_np, y_np)
    x_over = pd.DataFrame(x_np, columns=X_train.columns)
    y_over = pd.Series(y_np, name=y_train.name)
    return x_over, y_over

                             

In [ ]:
# %% Load and preprocess data
data_loader = DataLoader()
data_loader.load_dataset()
data_loader.preprocess_data()

In [ ]:
# Split the data for evaluation
X_train, X_test, y_train, y_test = data_loader.get_data_split()
print(X_train.shape)
print(X_test.shape)
X_train, y_train = data_loader.oversample(X_train, y_train)
print("After oversampling:", X_train.shape)

# Oversample the train data


(4088, 21)
(1022, 21)
After oversampling: (7778, 21)


In [ ]:
# %% Fit logistic regression model
lr = LogisticRegression(random_state=20, feature_names=X_train.columns, penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)
print("Training finished.")


Training finished.


In [ ]:
# %% Evaluate logistic regression model
y_pred = lr.predict(X_test)
print(f"F1 score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy {accuracy_score(y_test, y_pred)}")


F1 score 0.5138996389726957
Accuracy 0.7348336594911937


In [ ]:
# %% Explain local prediction
lr_local = lr.explain_local(X_test[:100], y_test[:100], name='Logistic Regression')
show(lr_local)


In [ ]:

# %% Fit decision tree model
tree = ClassificationTree()
tree.fit(X_train, y_train)
print("Training Finished")
y_pred = tree.predict(X_test)
print(f"F1 score {f1_score(y_test, y_pred, average='macro')}")
print(f"Accuracy{accuracy_score(y_test, y_pred)}")


Training Finished
F1 score 0.5240452498594523
Accuracy0.7544031311154599


In [ ]:
# %% Explain local prediction
tree_local = tree.explain_local(X_test[:100], y_test[:100], name='Tree')
show(tree_local)
